<h4>This notebook uses LGBM </h4>
<h5>I will keep updating the hyper-parameters</h5>
<font color='red'><h3>PLEASE DON'T FORGET TO UPVOTE</h3>

**Version 2**: A starter model </br>
**Version 3**:
Added 'u_in_cumsum'(refer to: https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/273974)

In [ ]:
import os
import glob
import shutil
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

pd.set_option('display.max_rows', 300)

In [ ]:
train = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
test = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')
sub = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')

In [ ]:
train['u_in_cumsum'] = (train['u_in']).groupby(train['breath_id']).cumsum()

train['last_value_u_in'] = train.groupby('breath_id')['u_in'].transform('last')
train['u_in_lag'] = train['u_in'].shift(1)
train['u_out_lag'] = train['u_out'].shift(1)
train = train.fillna(0)

train['u_in_diff'] = train['u_in'] - train['u_in_lag']
train['u_out_diff'] = train['u_out'] - train['u_out_lag']

train['breath_id__u_in__diffmax'] = train.groupby(['breath_id'])['u_in'].transform('max') - train['u_in']
train['breath_id__u_in__diffmean'] = train.groupby(['breath_id'])['u_in'].transform('mean') - train['u_in']

columns = [col for col in train.columns if col not in ['breath_id', 'pressure']]
X = train[columns]
Y = train['pressure']

In [ ]:
test['u_in_cumsum'] = (test['u_in']).groupby(test['breath_id']).cumsum()

test['last_value_u_in'] = test.groupby('breath_id')['u_in'].transform('last')
test['u_in_lag'] = test['u_in'].shift(1)
test['u_out_lag'] = test['u_out'].shift(1)
test = test.fillna(0)

test['u_in_diff'] = test['u_in'] - test['u_in_lag']
test['u_out_diff'] = test['u_out'] - test['u_out_lag']

test['breath_id__u_in__diffmax'] = test.groupby(['breath_id'])['u_in'].transform('max') - test['u_in']
test['breath_id__u_in__diffmean'] = test.groupby(['breath_id'])['u_in'].transform('mean') - test['u_in']

columns = [col for col in test.columns if col not in ['breath_id']]
t_x = test[columns]

In [ ]:
params = {'objective': 'regression',
          'learning_rate': 0.5,
          "boosting_type": "gbdt",
          "metric": 'mae',
          'n_jobs': -1}

In [ ]:
import lightgbm as lgb
from sklearn import metrics
from sklearn.model_selection import GroupKFold

n_splits = 5
preds = np.zeros((sub.shape[0], 1), dtype=np.float64)
gkfold = GroupKFold(n_splits=n_splits)
for i, (idx_train, idx_val) in enumerate(gkfold.split(X, Y, groups=train['breath_id'])):
    print(f'STARTING FOLD :- {i+1}')
    x_train, x_val, y_train, y_val = X.iloc[idx_train], X.iloc[idx_val], Y.iloc[idx_train], Y.iloc[idx_val]
    model = lgb.LGBMRegressor(**params, n_estimators=20000)
    model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_val, y_val)], verbose=100,
              early_stopping_rounds = 50,eval_metric='l2')
    print(metrics.mean_absolute_error(y_val, model.predict(x_val)))
    preds+=model.predict(t_x).reshape(-1, 1)
preds/=n_splits

In [ ]:
print(preds)

In [ ]:
sub['pressure'] = preds
sub.to_csv('submission.csv', index=False)
sub.head(10)